In [1]:
import pandas as pd
news_dataset = pd.read_csv('COVID Fake News Data.csv')
news_dataset.head()

,headlines,outcome
0,A post claims compulsory vacination violates t...,0
1,A photo claims that this person is a doctor wh...,0
2,Post about a video claims that it is a protest...,0
3,All deaths by respiratory failure and pneumoni...,0
4,The dean of the College of Biologists of Euska...,0


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
news_dataset = news_dataset.fillna('')
news_dataset['content'] = news_dataset['headlines']
print(news_dataset['content'])

0        A post claims compulsory vacination violates t...
1        A photo claims that this person is a doctor wh...
2        Post about a video claims that it is a protest...
3        All deaths by respiratory failure and pneumoni...
4        The dean of the College of Biologists of Euska...
                               ...                        
10196    A Chinese market caused the new coronavirus (v...
10197    The peak of the new coronavirus will happen in...
10198    Stores and supermarkets in Veracruz (Mexico) w...
10199    A chain message circulated on Tuesday, Jan. 14...
10200    Photo shows Muslims in Tamil Nadu state of Ind...
Name: content, Length: 10201, dtype: object


In [4]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [5]:
news_dataset['content'] = news_dataset['content'].apply(stemming)
print(news_dataset['content'])

0        post claim compulsori vacin violat principl bi...
1        photo claim person doctor die attend mani covi...
2        post video claim protest confin town aranda de...
3        death respiratori failur pneumonia regist covi...
4        dean colleg biologist euskadi state lot pcr fa...
                               ...                        
10196              chines market caus new coronaviru video
10197    peak new coronaviru happen two week jan two we...
10198    store supermarket veracruz mexico close due ne...
10199    chain messag circul tuesday jan warn peopl avo...
10200    photo show muslim tamil nadu state india float...
Name: content, Length: 10201, dtype: object


In [6]:
X = news_dataset['content'].values
y = news_dataset['outcome'].values

In [7]:
print(X)
print(y)

['post claim compulsori vacin violat principl bioethic coronaviru exist pcr test return mani fals posit influenza vaccin relat covid'
 'photo claim person doctor die attend mani covid patin hospit mu iz bueno air'
 'post video claim protest confin town aranda de duero burgo' ...
 'store supermarket veracruz mexico close due new coronaviru local govern ask peopl buy suppli'
 'chain messag circul tuesday jan warn peopl avoid go shangri la plaza mall mandaluyong citi suppos case sever acut respiratori syndrom sar report one mall medic clinic'
 'photo show muslim tamil nadu state india float lockdown rule ramzan offer night prayer']
[0 0 0 ... 0 0 0]


In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [9]:
voc_size=7000
corpus=X
print(corpus)

['post claim compulsori vacin violat principl bioethic coronaviru exist pcr test return mani fals posit influenza vaccin relat covid'
 'photo claim person doctor die attend mani covid patin hospit mu iz bueno air'
 'post video claim protest confin town aranda de duero burgo' ...
 'store supermarket veracruz mexico close due new coronaviru local govern ask peopl buy suppli'
 'chain messag circul tuesday jan warn peopl avoid go shangri la plaza mall mandaluyong citi suppos case sever acut respiratori syndrom sar report one mall medic clinic'
 'photo show muslim tamil nadu state india float lockdown rule ramzan offer night prayer']


In [10]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1013,
  1408,
  2025,
  4923,
  2814,
  1653,
  892,
  24,
  3749,
  3025,
  4558,
  3004,
  4662,
  1843,
  3012,
  5589,
  2887,
  6758,
  5144],
 [4166,
  1408,
  5313,
  2762,
  2301,
  410,
  4662,
  5144,
  2312,
  878,
  1952,
  1014,
  2493,
  5263],
 [1013, 4089, 1408, 4602, 3976, 3524, 4228, 5968, 6118, 855],
 [3066, 22, 5975, 2942, 6720, 5144, 712, 2882, 1202, 5982],
 [4409, 6282, 4524, 4672, 4282, 6651, 3025, 1843, 3012, 2077, 1608, 24],
 [1803,
  5144,
  6346,
  1483,
  406,
  49,
  3306,
  1328,
  264,
  5337,
  3517,
  3081,
  6133,
  3473,
  6973,
  3278,
  5670,
  2038,
  1393,
  6411,
  4317,
  5021,
  959],
 [1513, 5688, 6359, 1511, 6252, 24],
 [913, 1971, 2247, 856, 5144, 2002, 1971, 5058, 5140, 344, 3317, 6614, 5140],
 [1605,
  4332,
  1013,
  6096,
  4166,
  1213,
  1943,
  2385,
  2103,
  1577,
  4487,
  1753,
  2301,
  5144],
 [3680, 2171, 3110, 3799, 24, 4550],
 [564, 4164, 6864, 2702, 24, 1511, 1608],
 [4089, 4727, 3101, 2472, 2986, 5607, 1186, 4271, 6677],


In [11]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2887 6758 5144]
 [   0    0    0 ... 1014 2493 5263]
 [   0    0    0 ... 5968 6118  855]
 ...
 [   0    0    0 ... 2311 1572 6014]
 [   0    0    0 ... 6807 6673 1634]
 [   0    0    0 ... 6117 6157 4513]]


In [12]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       1013, 1408, 2025, 4923, 2814, 1653,  892,   24, 3749, 3025, 4558,
       3004, 4662, 1843, 3012, 5589, 2887, 6758, 5144])

In [45]:
embedding_vector_features=20
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.8))
model.add(LSTM(100))
model.add(Dropout(0.8))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 30, 20)            140000    
                                                                 
 dropout_14 (Dropout)        (None, 30, 20)            0         
                                                                 
 lstm_7 (LSTM)               (None, 100)               48400     
                                                                 
 dropout_15 (Dropout)        (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 188,501
Trainable params: 188,501
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
len(embedded_docs),y.shape

(10201, (10201,))

In [14]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [15]:
X_final.shape,y_final.shape

((10201, 30), (10201,))

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=0)

In [46]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
107/107 [==============================] - 15s 73ms/step - loss: 0.2563 - accuracy: 0.9473 - val_loss: 0.1846 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 7s 61ms/step - loss: 0.2037 - accuracy: 0.9530 - val_loss: 0.1855 - val_accuracy: 0.9546
Epoch 3/10
107/107 [==============================] - 6s 61ms/step - loss: 0.2007 - accuracy: 0.9530 - val_loss: 0.1851 - val_accuracy: 0.9546
Epoch 4/10
107/107 [==============================] - 6s 61ms/step - loss: 0.1929 - accuracy: 0.9530 - val_loss: 0.1699 - val_accuracy: 0.9546
Epoch 5/10
107/107 [==============================] - 7s 61ms/step - loss: 0.1439 - accuracy: 0.9579 - val_loss: 0.1299 - val_accuracy: 0.9635
Epoch 6/10
107/107 [==============================] - 6s 58ms/step - loss: 0.1099 - accuracy: 0.9682 - val_loss: 0.1183 - val_accuracy: 0.9658
Epoch 7/10
107/107 [==============================] - 6s 57ms/step - loss: 0.0946 - accuracy: 0.9726 - val_loss: 0.1146 - val_accuracy: 0.967

In [47]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

106/106 [==============================] - 3s 14ms/step


In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9673299673299673

In [49]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3202,   12],
       [  98,   55]], dtype=int64)

In [17]:
embedding_vector_features=20
model111=Sequential()
model111.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model111.add(Dropout(0.8))
model111.add(LSTM(100,return_sequences=True))
model111.add(Dropout(0.8))
model111.add(LSTM(100,return_sequences=False))
model111.add(Dense(1,activation='sigmoid'))
model111.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
print(model111.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 20)            140000    
                                                                 
 dropout (Dropout)           (None, 30, 20)            0         
                                                                 
 lstm (LSTM)                 (None, 30, 100)           48400     
                                                                 
 dropout_1 (Dropout)         (None, 30, 100)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 268,901
Trainable params: 268,901
Non-trai

In [18]:
model111.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
107/107 [==============================] - 128s 270ms/step - loss: 0.2213 - accuracy: 0.9511 - val_loss: 0.1848 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 22s 201ms/step - loss: 0.1897 - accuracy: 0.9530 - val_loss: 0.1882 - val_accuracy: 0.9546
Epoch 3/10
107/107 [==============================] - 22s 202ms/step - loss: 0.1897 - accuracy: 0.9530 - val_loss: 0.1823 - val_accuracy: 0.9546
Epoch 4/10
107/107 [==============================] - 22s 207ms/step - loss: 0.1504 - accuracy: 0.9582 - val_loss: 0.1128 - val_accuracy: 0.9673
Epoch 5/10
107/107 [==============================] - 22s 210ms/step - loss: 0.1007 - accuracy: 0.9685 - val_loss: 0.1101 - val_accuracy: 0.9691
Epoch 6/10
107/107 [==============================] - 23s 211ms/step - loss: 0.0892 - accuracy: 0.9704 - val_loss: 0.1068 - val_accuracy: 0.9709
Epoch 7/10
107/107 [==============================] - 22s 207ms/step - loss: 0.0787 - accuracy: 0.9747 - val_loss: 0.1160 - val_a

In [19]:
y_pred111 = model111.predict(X_test)
y_pred111 = np.round(y_pred111).astype(int)

106/106 [==============================] - 5s 24ms/step


In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred111)

0.9705969705969706

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred111)

array([[3207,    7],
       [  92,   61]], dtype=int64)

In [68]:
embedding_vector_features=20
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.9))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.9))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 30, 20)            140000    
                                                                 
 dropout_24 (Dropout)        (None, 30, 20)            0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 200)              96800     
 nal)                                                            
                                                                 
 dropout_25 (Dropout)        (None, 200)               0         
                                                                 
 dense_12 (Dense)            (None, 1)                 201       
                                                                 
Total params: 237,001
Trainable params: 237,001
Non-trainable params: 0
_______________________________________________

In [69]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
107/107 [==============================] - 22s 106ms/step - loss: 0.2695 - accuracy: 0.9426 - val_loss: 0.1849 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 8s 78ms/step - loss: 0.2069 - accuracy: 0.9530 - val_loss: 0.1851 - val_accuracy: 0.9546
Epoch 3/10
107/107 [==============================] - 8s 78ms/step - loss: 0.2067 - accuracy: 0.9530 - val_loss: 0.1826 - val_accuracy: 0.9546
Epoch 4/10
107/107 [==============================] - 8s 75ms/step - loss: 0.1889 - accuracy: 0.9529 - val_loss: 0.1466 - val_accuracy: 0.9546
Epoch 5/10
107/107 [==============================] - 8s 71ms/step - loss: 0.1572 - accuracy: 0.9564 - val_loss: 0.1466 - val_accuracy: 0.9587
Epoch 6/10
107/107 [==============================] - 6s 60ms/step - loss: 0.1404 - accuracy: 0.9603 - val_loss: 0.1284 - val_accuracy: 0.9620
Epoch 7/10
107/107 [==============================] - 8s 71ms/step - loss: 0.1348 - accuracy: 0.9634 - val_loss: 0.1330 - val_accuracy: 0.96

In [70]:
y_pred1 = model1.predict(X_test)
y_pred1 = np.round(y_pred1).astype(int)

106/106 [==============================] - 4s 16ms/step


In [67]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[3198,   16],
       [  89,   64]], dtype=int64)

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)


0.9667359667359667

In [32]:
from keras.layers import Conv1D, MaxPooling1D
embedding_vector_features=20
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Conv1D(200, kernel_size=3, activation = "relu"))
model2.add(MaxPooling1D())
model2.add(Dropout(0.8))
model2.add(LSTM(100)) 
model2.add(Dropout(0.8))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 30, 20)            140000    
                                                                 
 conv1d_3 (Conv1D)           (None, 28, 200)           12200     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 14, 200)          0         
 1D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 14, 200)           0         
                                                                 
 lstm_6 (LSTM)               (None, 100)               120400    
                                                                 
 dropout_9 (Dropout)         (None, 100)               0         
                                                      

In [33]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
107/107 [==============================] - 13s 49ms/step - loss: 0.2362 - accuracy: 0.9501 - val_loss: 0.1860 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 4s 37ms/step - loss: 0.1620 - accuracy: 0.9532 - val_loss: 0.1505 - val_accuracy: 0.9546
Epoch 3/10
107/107 [==============================] - 4s 36ms/step - loss: 0.0816 - accuracy: 0.9681 - val_loss: 0.1588 - val_accuracy: 0.9593
Epoch 4/10
107/107 [==============================] - 4s 36ms/step - loss: 0.0454 - accuracy: 0.9852 - val_loss: 0.1765 - val_accuracy: 0.9653
Epoch 5/10
107/107 [==============================] - 5s 44ms/step - loss: 0.0262 - accuracy: 0.9920 - val_loss: 0.1815 - val_accuracy: 0.9605
Epoch 6/10
107/107 [==============================] - 4s 39ms/step - loss: 0.0156 - accuracy: 0.9947 - val_loss: 0.2378 - val_accuracy: 0.9623
Epoch 7/10
107/107 [==============================] - 6s 53ms/step - loss: 0.0118 - accuracy: 0.9962 - val_loss: 0.2405 - val_accuracy: 0.959

In [34]:
y_pred2 = model2.predict(X_test)
y_pred2 = np.round(y_pred2).astype(int)

106/106 [==============================] - 2s 8ms/step


In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred2)

0.9652509652509652

In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred2)

array([[3199,   15],
       [ 102,   51]], dtype=int64)

In [24]:
from keras.layers import Conv1D, MaxPooling1D
embedding_vector_features=20
model3=Sequential()
model3.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model3.add(Conv1D(200, kernel_size=3, activation = "sigmoid"))
model3.add(MaxPooling1D())
model3.add(Dropout(0.6))
model3.add(LSTM(100,return_sequences=True)) 
model3.add(Dropout(0.6))
model3.add(LSTM(50,return_sequences=False)) 
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 30, 20)            140000    
                                                                 
 conv1d_1 (Conv1D)           (None, 28, 200)           12200     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 14, 200)          0         
 1D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 200)           0         
                                                                 
 lstm_3 (LSTM)               (None, 14, 100)           120400    
                                                                 
 dropout_5 (Dropout)         (None, 14, 100)           0         
                                                      

In [25]:
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
107/107 [==============================] - 63s 192ms/step - loss: 0.1979 - accuracy: 0.9511 - val_loss: 0.1900 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 7s 61ms/step - loss: 0.1914 - accuracy: 0.9530 - val_loss: 0.1852 - val_accuracy: 0.9546
Epoch 3/10
107/107 [==============================] - 8s 78ms/step - loss: 0.1914 - accuracy: 0.9530 - val_loss: 0.1849 - val_accuracy: 0.9546
Epoch 4/10
107/107 [==============================] - 8s 71ms/step - loss: 0.1900 - accuracy: 0.9530 - val_loss: 0.1850 - val_accuracy: 0.9546
Epoch 5/10
107/107 [==============================] - 8s 71ms/step - loss: 0.1894 - accuracy: 0.9530 - val_loss: 0.1835 - val_accuracy: 0.9546
Epoch 6/10
107/107 [==============================] - 6s 52ms/step - loss: 0.1575 - accuracy: 0.9532 - val_loss: 0.1337 - val_accuracy: 0.9587
Epoch 7/10
107/107 [==============================] - 7s 63ms/step - loss: 0.0896 - accuracy: 0.9694 - val_loss: 0.1309 - val_accuracy: 0.96

In [26]:
y_pred3 = model3.predict(X_test)
y_pred3 = np.round(y_pred3).astype(int)

106/106 [==============================] - 3s 7ms/step


In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred3)

0.9631719631719632

In [189]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred3)

array([[3181,   33],
       [  89,   64]], dtype=int64)

In [45]:
from keras.layers import Conv1D, MaxPooling1D
embedding_vector_features=20
model4=Sequential()
model4.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model4.add(Conv1D(200, kernel_size=3, activation = "sigmoid"))
model4.add(MaxPooling1D())
model4.add(Dropout(0.3))
model4.add(Bidirectional(LSTM(150)))
model4.add(Dropout(0.3))
model4.add(Dense(1,activation='sigmoid'))
model4.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model4.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 30, 20)            140000    
                                                                 
 conv1d_6 (Conv1D)           (None, 28, 200)           12200     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 14, 200)          0         
 1D)                                                             
                                                                 
 dropout_14 (Dropout)        (None, 14, 200)           0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 300)              421200    
 nal)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 300)              

In [46]:
model4.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
107/107 [==============================] - 19s 101ms/step - loss: 0.2023 - accuracy: 0.9456 - val_loss: 0.1858 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 10s 89ms/step - loss: 0.1899 - accuracy: 0.9530 - val_loss: 0.1850 - val_accuracy: 0.9546
Epoch 3/10
107/107 [==============================] - 8s 76ms/step - loss: 0.1834 - accuracy: 0.9530 - val_loss: 0.1567 - val_accuracy: 0.9546
Epoch 4/10
107/107 [==============================] - 8s 77ms/step - loss: 0.1093 - accuracy: 0.9625 - val_loss: 0.1225 - val_accuracy: 0.9647
Epoch 5/10
107/107 [==============================] - 9s 81ms/step - loss: 0.0629 - accuracy: 0.9767 - val_loss: 0.1414 - val_accuracy: 0.9641
Epoch 6/10
107/107 [==============================] - 8s 77ms/step - loss: 0.0427 - accuracy: 0.9858 - val_loss: 0.1375 - val_accuracy: 0.9638
Epoch 7/10
107/107 [==============================] - 8s 78ms/step - loss: 0.0316 - accuracy: 0.9899 - val_loss: 0.1556 - val_accuracy: 0.9

In [47]:
y_pred4 = model4.predict(X_test)
y_pred4 = np.round(y_pred4).astype(int)

106/106 [==============================] - 3s 17ms/step


In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred4)

0.9640629640629641

In [49]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred4)

array([[3184,   30],
       [  91,   62]], dtype=int64)

In [93]:
from tensorflow.keras.layers import GRU
embedding_vector_features=20
model5=Sequential()
model5.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model5.add(Dropout(0.7))
model5.add(GRU(200))
model5.add(Dropout(0.7))
model5.add(Dense(1,activation='sigmoid'))
model5.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
print(model5.summary())

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 30, 20)            140000    
                                                                 
 dropout_36 (Dropout)        (None, 30, 20)            0         
                                                                 
 gru_5 (GRU)                 (None, 200)               133200    
                                                                 
 dropout_37 (Dropout)        (None, 200)               0         
                                                                 
 dense_18 (Dense)            (None, 1)                 201       
                                                                 
Total params: 273,401
Trainable params: 273,401
Non-trainable params: 0
_________________________________________________________________
None


In [94]:
model5.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
107/107 [==============================] - 46s 89ms/step - loss: 0.2368 - accuracy: 0.9488 - val_loss: 0.1800 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 8s 73ms/step - loss: 0.1662 - accuracy: 0.9533 - val_loss: 0.1404 - val_accuracy: 0.9549
Epoch 3/10
107/107 [==============================] - 8s 73ms/step - loss: 0.1109 - accuracy: 0.9634 - val_loss: 0.1169 - val_accuracy: 0.9673
Epoch 4/10
107/107 [==============================] - 8s 74ms/step - loss: 0.0878 - accuracy: 0.9707 - val_loss: 0.1224 - val_accuracy: 0.9661
Epoch 5/10
107/107 [==============================] - 8s 75ms/step - loss: 0.0756 - accuracy: 0.9756 - val_loss: 0.1166 - val_accuracy: 0.9688
Epoch 6/10
107/107 [==============================] - 8s 74ms/step - loss: 0.0550 - accuracy: 0.9829 - val_loss: 0.1244 - val_accuracy: 0.9673
Epoch 7/10
107/107 [==============================] - 6s 54ms/step - loss: 0.0540 - accuracy: 0.9833 - val_loss: 0.1363 - val_accuracy: 0.967

In [95]:
y_pred5 = model5.predict(X_test)
y_pred5 = np.round(y_pred5).astype(int)

106/106 [==============================] - 3s 19ms/step


In [96]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred5)

0.9682209682209683

In [97]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred5)

array([[3199,   15],
       [  92,   61]], dtype=int64)

In [64]:
from keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import GRU
embedding_vector_features=20
model7=Sequential()
model7.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model7.add(Conv1D(200, kernel_size=3, activation = "sigmoid"))
model7.add(MaxPooling1D())
model7.add(Dropout(0.6))
model7.add(GRU(200))
model7.add(Dropout(0.6))
model7.add(Dense(1,activation='sigmoid'))
model7.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model7.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 30, 20)            140000    
                                                                 
 conv1d_11 (Conv1D)          (None, 28, 200)           12200     
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 14, 200)          0         
 g1D)                                                            
                                                                 
 dropout_23 (Dropout)        (None, 14, 200)           0         
                                                                 
 gru_3 (GRU)                 (None, 200)               241200    
                                                                 
 dropout_24 (Dropout)        (None, 200)               0         
                                                     

In [65]:
model7.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
107/107 [==============================] - 13s 74ms/step - loss: 0.2159 - accuracy: 0.9492 - val_loss: 0.1885 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 8s 70ms/step - loss: 0.2005 - accuracy: 0.9530 - val_loss: 0.1868 - val_accuracy: 0.9546
Epoch 3/10
107/107 [==============================] - 6s 59ms/step - loss: 0.1851 - accuracy: 0.9530 - val_loss: 0.1607 - val_accuracy: 0.9546
Epoch 4/10
107/107 [==============================] - 6s 51ms/step - loss: 0.1145 - accuracy: 0.9622 - val_loss: 0.1202 - val_accuracy: 0.9667
Epoch 5/10
107/107 [==============================] - 5s 51ms/step - loss: 0.0668 - accuracy: 0.9772 - val_loss: 0.1407 - val_accuracy: 0.9679
Epoch 6/10
107/107 [==============================] - 6s 52ms/step - loss: 0.0426 - accuracy: 0.9848 - val_loss: 0.1532 - val_accuracy: 0.9667
Epoch 7/10
107/107 [==============================] - 6s 52ms/step - loss: 0.0351 - accuracy: 0.9873 - val_loss: 0.1500 - val_accuracy: 0.965

In [66]:
y_pred7 = model7.predict(X_test)
y_pred7 = np.round(y_pred7).astype(int)

106/106 [==============================] - 3s 14ms/step


In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred7)

0.9631719631719632

In [68]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred7)

array([[3177,   37],
       [  87,   66]], dtype=int64)

In [79]:
from tensorflow.keras.layers import GRU
embedding_vector_features=20
model9=Sequential()
model9.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model9.add(Dropout(0.8))
model9.add(GRU(300,return_sequences=True))
model9.add(Dropout(0.8))
model9.add(GRU(100,return_sequences=False))
model9.add(Dense(1,activation='sigmoid'))
model9.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
print(model9.summary())

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 30, 20)            140000    
                                                                 
 dropout_29 (Dropout)        (None, 30, 20)            0         
                                                                 
 gru_6 (GRU)                 (None, 30, 300)           289800    
                                                                 
 dropout_30 (Dropout)        (None, 30, 300)           0         
                                                                 
 gru_7 (GRU)                 (None, 100)               120600    
                                                                 
 dense_14 (Dense)            (None, 1)                 101       
                                                                 
Total params: 550,501
Trainable params: 550,501
Non-t

In [80]:
model9.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
107/107 [==============================] - 39s 281ms/step - loss: 0.2131 - accuracy: 0.9504 - val_loss: 0.1722 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 27s 254ms/step - loss: 0.1595 - accuracy: 0.9543 - val_loss: 0.1299 - val_accuracy: 0.9611
Epoch 3/10
107/107 [==============================] - 27s 253ms/step - loss: 0.1163 - accuracy: 0.9640 - val_loss: 0.1064 - val_accuracy: 0.9685
Epoch 4/10
107/107 [==============================] - 27s 250ms/step - loss: 0.1005 - accuracy: 0.9685 - val_loss: 0.1115 - val_accuracy: 0.9691
Epoch 5/10
107/107 [==============================] - 27s 253ms/step - loss: 0.0915 - accuracy: 0.9719 - val_loss: 0.1062 - val_accuracy: 0.9691
Epoch 6/10
107/107 [==============================] - 27s 251ms/step - loss: 0.0760 - accuracy: 0.9767 - val_loss: 0.1017 - val_accuracy: 0.9715
Epoch 7/10
107/107 [==============================] - 27s 257ms/step - loss: 0.0697 - accuracy: 0.9786 - val_loss: 0.1118 - val_ac

In [81]:
y_pred9 = model9.predict(X_test)
y_pred9 = np.round(y_pred9).astype(int)

106/106 [==============================] - 8s 64ms/step


In [82]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred9)

0.9694089694089694

In [83]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred9)

array([[3211,    3],
       [ 100,   53]], dtype=int64)

In [84]:
embedding_vector_features=20
model10=Sequential()
model10.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model10.add(Dropout(0.9))
model10.add(Bidirectional(LSTM(100,return_sequences=True)))
model10.add(Dropout(0.9))
model10.add(Bidirectional(LSTM(50,return_sequences=False)))
model10.add(Dense(1,activation='sigmoid'))
model10.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model10.summary())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 30, 20)            140000    
                                                                 
 dropout_31 (Dropout)        (None, 30, 20)            0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 30, 200)          96800     
 nal)                                                            
                                                                 
 dropout_32 (Dropout)        (None, 30, 200)           0         
                                                                 
 bidirectional_6 (Bidirectio  (None, 100)              100400    
 nal)                                                            
                                                                 
 dense_15 (Dense)            (None, 1)               

In [85]:
model10.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
107/107 [==============================] - 35s 173ms/step - loss: 0.2309 - accuracy: 0.9483 - val_loss: 0.1938 - val_accuracy: 0.9546
Epoch 2/10
107/107 [==============================] - 14s 131ms/step - loss: 0.1912 - accuracy: 0.9530 - val_loss: 0.1822 - val_accuracy: 0.9546
Epoch 3/10
107/107 [==============================] - 14s 130ms/step - loss: 0.1732 - accuracy: 0.9514 - val_loss: 0.1462 - val_accuracy: 0.9546
Epoch 4/10
107/107 [==============================] - 14s 129ms/step - loss: 0.1479 - accuracy: 0.9562 - val_loss: 0.1560 - val_accuracy: 0.9554
Epoch 5/10
107/107 [==============================] - 14s 133ms/step - loss: 0.1317 - accuracy: 0.9611 - val_loss: 0.1328 - val_accuracy: 0.9593
Epoch 6/10
107/107 [==============================] - 14s 130ms/step - loss: 0.1245 - accuracy: 0.9646 - val_loss: 0.1230 - val_accuracy: 0.9632
Epoch 7/10
107/107 [==============================] - 14s 130ms/step - loss: 0.1106 - accuracy: 0.9661 - val_loss: 0.1099 - val_ac

In [86]:
y_pred10 = model9.predict(X_test)
y_pred10 = np.round(y_pred10).astype(int)

106/106 [==============================] - 7s 63ms/step


In [87]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred10)

0.9694089694089694

In [78]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred10)

array([[3197,   17],
       [  90,   63]], dtype=int64)